# "Classifiez automatiquement des biens de consommation"
_NLP Cleaning Notebook_

In [3]:
import pandas as pd
df = pd.DataFrame()
for i in range (10):
    df['r'+str(i)] = ['u']*20

## 0 Preliminaries

### 0.0 Importing Packages and Modules

Checking whether the notebook is on Colab or PC

In [4]:
import sys
is_colab = 'google.colab' in sys.modules
is_colab, sys.executable

(True, '/usr/bin/python3')

Mounting my Drive if on Colab

In [5]:
if is_colab==True:
    from google.colab import files, output, drive
    drive.mount('/gdrive')
    %cd /gdrive
    print("You're on Google Colab")
else:
    print("You're on a PC")

Mounted at /gdrive
/gdrive
You're on Google Colab


Installations and importations required in the virtual environment.

In [6]:
import os
if os.getcwd()!='/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS':
    os.chdir('/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS')

In [7]:
!pip install -r requirements_pict.txt

     |████████████████████████████████| 81kB 2.5MB/s 
     |████████████████████████████████| 92kB 5.6MB/s 
     |████████████████████████████████| 276kB 17.2MB/s 
     |████████████████████████████████| 143kB 20.2MB/s 
     |████████████████████████████████| 1.1MB 19.1MB/s 
  Created wheel for gTTS-token: filename=gTTS_token-1.1.3-cp36-none-any.whl size=4096 sha256=1ed909c21241ca645d465be68b38e550840681073c00168a8c27efbaa25e6a55
  Stored in directory: /root/.cache/pip/wheels/dd/11/61/33f7e51bf545e910552b2255eead2a7cd8ef54064b46dceb34
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861418 sha256=36080461329e47914c5d2eecdc6116c8d715bef5395659f7ed8710e99235b71c
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
  Created wheel for mwparserfromhell: filename=mwparserfromhell-0.5.4-cp36-cp36m-linux_x86_64.whl size=183771 sha256=8ac66ff3c071eab803881553ee665b7b02d54e6fafac9449aa2cb0078857f7e

In [8]:
from P6_functions import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Installations (creating the requirements file)

In [9]:
# !pip install gtts
# !pip install wikipedia2vec==0.2.2
# !pip install opencv-python==3.4.2.17
# !pip install opencv-contrib-python==3.4.2.17
# !pip install category_encoders
# !pip install wikipedia2vec==0.2.2
# !pip install utils

In [10]:
!pip freeze > requirements_pict.txt

Importation of modules and packages. 

In [11]:
import io
import string
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor']='w'
import PIL
import cv2
from scipy import ndimage
from scipy.spatial import distance

# import warnings
# warnings.filterwarnings('ignore')

Setting pandas display options.

In [12]:
dictPdSettings = {'display.max_rows': 500, 'display.width': 100,
                  'display.max_colwidth': 100,
                  'display.float_format': lambda x: '%.2f' % x}
for k,v in dictPdSettings.items():
  pd.set_option(k,v)

To play audio text-to-speech during execution.

In [13]:
from IPython.display import Audio
from gtts import gTTS

def speak(text, lang='en'):
    with io.BytesIO() as f:
        gTTS(text=text, lang=lang).write_to_fp(f)
        f.seek(0)
        return Audio(f.read(), autoplay=True)

In [14]:
# speak('Packages and modules successfully imported')

### 0.1 Importing the datasets

Data is composed of 9 distinct .csv files we'll load in a dictionnary of dataframes.

In [15]:
df = pd.read_csv("../DATA/flipkart_com-ecommerce_sample_1050.csv",
                 sep=',', 
                 index_col = 'uniq_id',
                 encoding ='utf-8')
print("-----> Importation of .csv in the notebook: OK")

-----> Importation of .csv in the notebook: OK


In [16]:
# speak('Datasets successfully imported')

### 0.2 Categories

In [17]:
# Converting the strings in 'product_category_tree' column in 6 categ columns

# determining the maximum tree depth of categories
ser_depth = df['product_category_tree'].apply(lambda x: x.count('>>'))
max_depth = ser_depth.max()

def str_cleaning(ind, my_str, name_level_cols):
    my_str = my_str.replace("[\"", "").replace("\"]", "")
    tab_str = my_str.split(">>")
    size_tab_str = len(tab_str)
    tup_str = tuple([tab_str[i].strip() if i<size_tab_str else "" \
                     for i in np.arange(max_depth) ])
    return tup_str

name_level_cols = ['cat_level_'+str(i) for i in np.arange(max_depth)]
ser_tuple = df['product_category_tree']\
    .apply(lambda s: str_cleaning(s.index, s, name_level_cols))
df_cat_level = pd.DataFrame([[a,'/'.join([a,b]),'/'.join([a,b,c]),
                              '/'.join([a,b,c,d]),'/'.join([a,b,c,d,e]),
                              '/'.join([a,b,c,d,e,f])] \
                             for a,b,c,d,e,f in ser_tuple.values],
                            columns=name_level_cols, index=df.index)

In [18]:
# Create a dataframe for images

df_image = df[['product_name', 'description']].copy('deep')
df_image['category'] = \
    df_cat_level['cat_level_0'].replace({'Home Furnishing': 'Furnishing',
                                         'Baby Care': 'Baby', 
                                         'Watches': 'Watches',
                                         'Home Decor & Festive Needs': 'Decor',
                                         'Kitchen & Dining': 'Kitchen',
                                         'Beauty and Personal Care': 'Beauty',
                                         'Computers': 'Computers'})

df_image.index = [s+'.jpg' for s in df_image['category'].index]    
indexes = df_image.index

### 0.3 Images properties

In [19]:
from PIL import Image
from IPython.display import Image as ShowImage

In [20]:
from google.colab.patches import cv2_imshow

### 0.4 Browsing images

## FUNCTIONS

#### preproc_image

In [21]:
from math import ceil
from PIL import Image
import colorsys

In [47]:
def print_thumbnails(file_path, li_files, sq_size=224, n_rows=1,
                     figsize=(15,2), fig=None):

    n_tot = len(li_files)
    n_cols = (n_tot//n_rows)+((n_tot%n_rows)>0)*1
    fig = plt.figure(figsize=figsize) if fig is None else fig
    for i, ind in enumerate(li_files,1):

        img = np.array(Image.open(file_path+ind))
        img = preproc_image(img, size=size, fill_col=(255,255,255),
                        interpolation=Image.ANTIALIAS)
        ax = fig.add_subplot(n_rows,n_cols,i)
        # ax = fig.add_subplot(1,(len(li_files)),i)
        ax.imshow(img)
        ax.set_axis_off()

In [23]:
''' Takes an image, resizes the image and fills the non existing space
with custom color 
'''

import cv2
import numpy as np
from PIL import Image, ImageOps
from PIL.ImageFilter import GaussianBlur

def preproc_image(img, size=224, fill_col=(255,255,255),
                  autocontrast = False, equalize=False,
                  gauss_blur = None, interpolation=cv2.INTER_AREA):

    img = Image.fromarray(img)
    if autocontrast:
        img = ImageOps.autocontrast(img)
    if equalize:
        img = ImageOps.equalize(img)
    if gauss_blur is not None:
        img = img.filter(GaussianBlur(radius=gauss_blur))

    w, h = img.size
    if h == w:
        new_img = img
    else:
        dif = h if h > w else w
        new_img = Image.new('RGB', (dif, dif), fill_col)
        new_img.paste(img, (int((dif - w) / 2), int((dif - h) / 2)))

    return cv2.resize(np.asarray(new_img), (size, size), interpolation)


In [56]:
# rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)
file_path = "../DATA/Images/"
preproc_params={'sq_size': 80}
sq_size = 224
figsize = (10,2)
n_img = 20
sercat = df_image('category')

def print_sample_by(file_path, sercat, n_img=10, sq_size,
                    figsize(20,2), **preproc_params)
gb = df_image.groupby(sercat)
for name, sub_df in gb:
    li_files = sub_df.sample(n_img).index
    print(name)
    print_thumbnails(file_path, li_files,
                     figsize = figsize,
                     sq_size=sq_size, n_rows=2)
    plt.show()

SyntaxError: ignored

In [ ]:
   dict_images = {}

   # dict_images[ind] = img
   
    # img_ravel = np.stack(np.array(rgb_to_hsv(img[:,:,0],
    #                                          img[:,:,1],
    #                                          img[:,:,2])), 0).ravel()
    # ser = pd.Series(img_ravel, index = li_cols, name=ind).to_frame()
    # df_temp = pd.concat([df_temp, ser.T], axis=0)

# df_image = pd.concat([df_image, df_temp], axis=1)

In [ ]:
def show_image_and_histogram(img, img_orig=np.array([]), filter_name=''):
    if img_orig.any():
        f = plt.figure(figsize=(15, 10))
        ax1 = f.add_subplot(221)
        ax2 = f.add_subplot(222)
        ax3 = f.add_subplot(223)
        ax4 = f.add_subplot(224)
        ax1.imshow(img_orig, cmap='gray')
        ax1.set_title('Original image')
        ax2.hist(img_orig.flatten(), bins=range(256))
        ax2.set_title('Original histogram')
        ax3.imshow(img, cmap='gray')
        ax3.set_title(f'Image after {filter_name}')
        ax4.hist(img.flatten(), bins=range(256))
        ax4.set_title(f'Histogram after {filter_name}')
    else:
        f = plt.figure(figsize=(15, 5))
        ax1 = f.add_subplot(121)
        ax2 = f.add_subplot(122)
        ax1.imshow(img, cmap='gray')
        ax1.set_title(f'Image after {filter_name}')
        ax2.hist(img.flatten(), bins=range(256))
        ax2.set_title(f'Histogram after {filter_name}')

    plt.show()

In [ ]:
def load_thumbnail(path, ):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (500, 375))
    return img

In [70]:
%pwd

'/gdrive/My Drive/--DATA SCIENCE/PROJET6/NOTEBOOKS'

In [ ]:
 ('../DATA/Images/'+im_name, )

In [ ]:
im_name = df_image['category'].index[-1]
img = cv2.imread( '../DATA/Images/'+im_name)
cv2_imshow(img)

In [75]:
# gray conversion
img = load_image('Images/n02085782-Japanese_spaniel/n02085782_1855.jpg')
print("Size: {}".format(img.shape))
show_image_and_histogram(img, filter_name='gray conversion')

NameError: ignored

In [ ]:
# equalization
img_orig = load_image('Images/n02085782-Japanese_spaniel/n02085782_1855.jpg')
img = cv2.equalizeHist(img_orig)
show_image_and_histogram(img, img_orig, filter_name='equalization')

In [ ]:
# average filter
img_orig = load_image('Images/n02085782-Japanese_spaniel/n02085782_1855.jpg')

show_images([
    (img_orig, 'Original image'), 
    (cv2.blur(img_orig, (3, 3)), 'Average filter - size 3x3'), 
    (cv2.blur(img_orig, (7, 7)), 'Average filter - size 7x7'), 
    (cv2.blur(img_orig, (15, 15)), 'Average filter - size 15x15')
])

In [ ]:
# gaussian filter
img_orig = load_image('images/n02085782-Japanese_spaniel/n02085782_1855.jpg')

show_images([
    (img_orig, 'Original image'), 
    (cv2.GaussianBlur(img_orig, (3, 3), 0), 'Gaussian filter - size 3x3'), 
    (cv2.GaussianBlur(img_orig, (7, 7), 0), 'Gaussian filter - size 7x7'), 
    (cv2.GaussianBlur(img_orig, (15, 15), 0), 'Gaussian filter - size 15x15')
])

In [ ]:
# median filter
img_orig = load_image('images/n02085782-Japanese_spaniel/n02085782_1855.jpg')
img = cv2.fastNlMeansDenoising(img_orig)
show_images([
    (img_orig, 'Original image'),
    (img, 'Non-locals means filter')
])

In [ ]:
# non local means
show_images([
    (img_orig, 'Original image'), 
    (cv2.medianBlur(img_orig, 3), 'Median filter - size 3x3'), 
    (cv2.medianBlur(img_orig, 7), 'Median filter - size 7x7'), 
    (cv2.medianBlur(img_orig, 15), 'Median filter - size 15x15')
])

In [ ]:
# SIFT features extraction
img_orig = load_image('images/n02085620-Chihuahua/n02085620_11696.jpg')
sift = cv2.xfeatures2d.SIFT_create()
kp, desc = sift.detectAndCompute(img_orig, None)

f, ax = plt.subplots(figsize = (15, 8))
ax.imshow(cv2.drawKeypoints(img_orig, kp, img_orig.copy()))
plt.show()

## 1 Image pre-processing

### 1.0 Vectorization, resizing, adjusting contrast of images

Load each image, resizes to 224*224 and store the values of pixel in HSV in the dataframe

In [56]:
# from PIL import Image, ImageOps

# # to silence the decompression bomb warning
# Image.MAX_IMAGE_PIXELS = 1000000000        
# import colorsys

# rgb_to_hsv = np.vectorize(colorsys.rgb_to_hsv)

# size = 224

# # df_temp = pd.DataFrame([])
# dict_images = {}
# li_cols = [ l+'_'+str(i) for l in ['H', 'S', 'V']\
#            for i in range(size*size)]

# for i, ind in enumerate(indexes):
#     img = np.array(Image.open("../DATA/Images/"+ind+".jpg"))
#     img = preproc_image(img, size=224, fill_col=(255,255,255),
#                         autocontrast = True, equalize=False,
#                         gauss_blur=3, interpolation=Image.ANTIALIAS)
#     dict_images[ind] = img
#     # img_ravel = np.stack(np.array(rgb_to_hsv(img[:,:,0],
#     #                                          img[:,:,1],
#     #                                          img[:,:,2])), 0).ravel()
#     # ser = pd.Series(img_ravel, index = li_cols, name=ind).to_frame()
#     # df_temp = pd.concat([df_temp, ser.T], axis=0)

# # df_image = pd.concat([df_image, df_temp], axis=1)

In [ ]:
# import dill
# dill.dump(dict_images, open('dict_images.pkl', mode='wb'))

In [57]:
import dill
df_image = dill.load(open('df_image.pkl', mode='rb'))
dict_images = dill.load(open('dict_images.pkl', mode='rb'))

Visualisation séparée des trois canaux (Hue, Saturation, Value)

In [ ]:
# fig = plt.figure(figsize=(10,4))
# li_n = []
# for im, color, title, i in zip([img_h, img_s, img_v],
#                                 ['red', 'blue', 'black'],
#                                 ['Hue', 'Saturation', 'Value'],
#                                 range(1,4)):
#     ax1 = fig.add_subplot(2,3,i)
#     ax1.imshow(im, cmap='Greys')
#     ax1.set(xlim=(0,255))
#     ax1.set_title(title, fontweight='bold')
#     ax2 = fig.add_subplot(2,3,i+3)
#     n, bins, patches = ax2.hist(im.flatten(), color=color, bins=range(256))
#     li_n.append(n)
# plt.tight_layout(rect=[0,0,1,0.92])
# plt.show()

## 2 Clustering

In [ ]:
df_res_clust = pd.DataFrame()
df_res_clust['category'] = df_image['category']

### Clustering on whole resized HSV images (pixels)

In [ ]:
# Data selection to train clusterer (HSV 224x224 pixels)
X_df = df_image.iloc[:,3:]
print("Size of the training data :", X_df.shape)

In [ ]:
# Visualization of the images (HSV 224 x 224 pixels) on a t-SNE projection

ser_categories = pd.Series(df_image['category'].values, 
                           index=df_image.index,
                           name='Clust')

plot_projection(X_df, model=None, ser_clust=ser_categories,
                proj='t-SNE', size=10, legend_on=True,
                title="Projection t-SNE of the pictures\n(HSV 224x224 pixels)",
                figsize=(10, 6), palette='tab10',
                fig=None, ax=None, random_state=14)

In [ ]:
# Create the model
from sklearn.cluster import KMeans
k=7 # number of categories
km = KMeans(n_clusters=k) 

In [ ]:
# Fitting the Kmeans model
km.fit(X_df)

In [ ]:
# Appending the best results of the Kmeans clustering
df_res_clust['KMclust_HSVpx'] = km.labels_

In [ ]:
df_res_clust.sample(5)

### Clustering on a bag of visual words (SIFT)

In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans

### Extracts local features from images using SIFT

The below function returns an array whose first index holds a list that holds all local features from all images without an order. This is our visual dictionary. And the second index holds the sift vectors dictionary which holds the descriptors but this is separated class by class

In [ ]:
# Creates descriptors using sift 
# Takes one parameter that is images dictionary
# Return an array whose first index holds the decriptor_list without an order
# And the second index holds the sift_vectors dictionary which holds
# the descriptors but this is seperated class by class

def sift_features(images):
    dict_sift_descriptors, dict_keypoints = {}, {}
    all_descriptors_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for key,img in images.items():
        features = []
        kp, des = sift.detectAndCompute(img,None)
        all_descriptors_list.extend(des)
        dict_sift_descriptors[key] = des
        dict_keypoints[key] = kp
    return dict_keypoints, dict_sift_descriptors, all_descriptors_list

# def get_descriptors(image_path):
#     # load image and convert it to grayscale
#     img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#     # detect key points and descriptors
#     keypoints, descriptors = model.detectAndCompute(img_gray, None)
#     # returns keypoints and descriptors
#     return keypoints, descriptors

In [ ]:
# # Store descriptors and bovw features 
# # keypoints_list, descriptor_list, all_bovw_feature = sift_features(dict_images) 
# dict_keypoints, dict_sift_descriptors, all_descriptors_list = \
#                                              sift_features(dict_images) 

In [ ]:
# import dill
# dill.dump((dict_sift_descriptors, all_descriptors_list),
#           open("descriptors.pkl", "wb"))

In [ ]:
import dill
dict_sift_descriptors, all_descriptors_list  = \
    dill.load(open("descriptors.pkl", "rb"))

In [ ]:
# print("list of the keypoints", len(keypoints_list))
print("list of all the descriptors: ", len(all_descriptors_list))
print("dictionnary giving all the descriptors (vectors) for each image: ",
      len(dict_sift_descriptors.keys()))

### Selects the visual words

Send the visual dictionary to the k-means clustering algorithm and find the visual words which are center points.

In [ ]:
# A k-means clustering algorithm who takes 2 parameter which is number 
# of cluster(k) and the other is descriptors list(unordered 1d array)
# Returns an array that holds central points.

from sklearn.cluster import KMeans

def descriptors_cluster_centers(k, descriptors_list):
    clusterer = KMeans(n_clusters = k, n_init=10)
    clusterer.fit(descriptors_list)
    visual_words = clusterer.cluster_centers_ 
    return visual_words, clusterer

In [ ]:
# # Takes the central points which is visual words   

# visual_words, desc_clusterer = descriptors_cluster_centers(150,
#                                                            all_descriptors_list)
# visual_words.shape

In [ ]:
# import dill
# dill.dump((visual_words, desc_clusterer), open("visual_words.pkl", "wb"))

In [ ]:
import dill
visual_words, desc_clusterer = dill.load(open("visual_words.pkl", "rb"))

In [ ]:
# détermine pour chaque image a quel cluster appartient les descripteurs
# renvoie un dictionnaire avec la liste des numéros des clusters parmi 150
# convertit le dictionnaire en pd.Series, puis en dataframe (multilabelbinarizer)
vw_list = []
ind_img_list = []
for n_img, desc_list in dict_sift_descriptors.items():
    vwords = desc_clusterer.predict(desc_list) # liste des numéros des visuals words de l'image
    vw_list.append(vwords) # liste des listes de vw pour toutes les images
    ind_img_list.append(n_img) # liste des indexes des images

In [ ]:
from collections import Counter
df_visual_words = pd.DataFrame([Counter(x) for x in vw_list],
                               index = ind_img_list).fillna(0)
df_visual_words = df_visual_words[sorted(df_visual_words.columns)]

In [ ]:
df_visual_words

### KMeans clustering on the BoVW

In [ ]:
# Data selection to train clusterer (HSV 224x224 pixels)
X_df = df_visual_words
print("Size of the training data :", X_df.shape)

In [ ]:
# Visualization of the images (BoVW of 150 VW) on a t-SNE projection

ser_categories = pd.Series(df_image['category'].values, 
                           index=df_image.index,
                           name='Clust')

plot_projection(X_df, model=None, ser_clust=ser_categories,
                proj='t-SNE', size=10, legend_on=True,
                title="Projection t-SNE of the pictures\n(Bag of 150 VirtualWords)",
                figsize=(10, 6), palette='tab10',
                fig=None, ax=None, random_state=14)

In [99]:
# Create the model
from sklearn.cluster import KMeans
k=7 # number of categories
km = KMeans(n_clusters=k) 

In [ ]:
# Fitting the Kmeans model
km.fit(X_df)

In [ ]:
# Appending the best results of the Kmeans clustering
df_res_clust['KMclust_Bo150VW'] = km.labels_

In [ ]:
df_res_clust.sample(5)

### Feature extraction par transfer learning sur réseau préentrainé

## 3 Comparison of clusters labels with true categories

# Exportation

Now we export the dataset of aggregated orders in a .csv file.